In [ ]:
import os
import subprocess
from collections import defaultdict

import pandas as pd
from Bio import SeqIO

In [ ]:
def calculate_depth(assembly_id):
    reference_fasta = os.path.join(reference_genome_path, f'{assembly_id}.fasta')
    depth_file = os.path.join(depth_file_path, f"{assembly_id}.depth")
    
    with open(reference_fasta, "r") as handle:
        genome_length = 0
        for record in SeqIO.parse(handle, "fasta"):
            genome_length += len(record.seq)

    genome_pos_count = 0
    genome_totol_count = 0

    with open(depth_file, "r") as depth:
        for line in depth.readlines():

            pos = int(line.split("\t")[1])
            depth = int(line.strip().split("\t")[2])

            if depth >= min_depth:
                genome_pos_count += 1
                genome_totol_count += depth

    breadth_coverage = genome_pos_count/genome_length
    depth_coverage = genome_totol_count/genome_pos_count
    
    return breadth_coverage, depth_coverage

In [ ]:
reference_metadata = pd.read_csv('reference_metadata.csv')
reference_genome_path = 'reference_genomes'
depth_file_path = 'alignment/depth_files'

In [ ]:
downloaded_assemblies = reference_metadata[reference_metadata['Downloaded']]

breadth_coverage_list = []
depth_coverage_list = []
for assembly_id in downloaded_assemblies['Assembly Accession ID']:
    breadth_coverage, depth_coverage = calculate_depth(assembly_id)
    breadth_coverage_list.append(breadth_coverage)
    depth_coverage_list.append(depth_coverage)
    

In [ ]:
downloaded_assemblies['Breadth Coverage'] = breadth_coverage_list
downloaded_assemblies['Depth Coverage'] = depth_coverage_list

In [ ]:
downloaded_assemblies.to_csv('coverage.csv', index=False)

In [ ]:
downloaded_assemblies